In [15]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
def get_bound(index1, index2, bonds):
    bond = frozenset({tuple(index1),tuple(index2)})
    if bond in bonds:
        return bonds[bond]
    else:
        bonds[bond] = np.random.standard_normal()
        return bonds[bond]

In [3]:
def increase_entry_by_one(A, j):
    A_modified = A.copy()
    A_modified[j] += 1
    return A_modified
def decrease_entry_by_one(A, j):
    A_modified = A.copy()
    A_modified[j] -= 1
    return A_modified

In [4]:
def get_neighbor(indices,L):
    neighbor_index = []
    for j in range(len(L)):
        if (indices[j] == 0):
            neighbor_index.append(increase_entry_by_one(indices, j))
            indice_copy = indices.copy()
            indice_copy[j] = L[j]-1
            neighbor_index.append(indice_copy)
        elif (indices[j] == L[j]-1):
            neighbor_index.append(decrease_entry_by_one(indices, j))
            indice_copy = indices.copy()
            indice_copy[j] = 0
            neighbor_index.append(indice_copy)
        else:
            neighbor_index.append(increase_entry_by_one(indices, j))
            neighbor_index.append(decrease_entry_by_one(indices, j))
    return neighbor_index

In [5]:
def get_energy(spin, spin_index, neighbor_index, S):
    energy = 0
    for neighbor in neighbor_index:
        bond = get_bound(spin_index, neighbor, bonds)
        energy = energy + bond*spin*S[tuple(neighbor)]
    return energy

In [6]:
def overlap(S1,S2,N):
    return np.sum(S1*S2)/N

def sweep(S,L,N):
    sweep = 0
    while sweep < N:
        indices = [np.random.choice(dim) for dim in L]
        spin = S[tuple(indices)]
        neighbor_index = get_neighbor(indices,L)
        beforeE = get_energy(spin, indices, neighbor_index, S)
        afterE = get_energy(-spin, indices, neighbor_index, S)
        deltaE = afterE - beforeE
        if deltaE > 0:
            S[tuple(indices)] = -spin
        sweep = sweep+1

In [7]:
def is_active(index, S, L):
    spin = S[tuple(index)]
    neighbor_index = get_neighbor(np.asarray(index),L)
    beforeE = get_energy(spin, index, neighbor_index, S)
    afterE = get_energy(-spin, index, neighbor_index, S)
    deltaE = afterE - beforeE
    return deltaE > 0

In [8]:
def get_active(S,L):
    it = np.nditer(S,flags = ['multi_index'])
    active_indices = []
    while not it.finished:
        index = it.multi_index
        if is_active(index, S, L):
            active_indices.append(index)
        it.iternext()
    return active_indices

In [9]:
import random

def kineticMonteCarlo(S,L,active_list):
    l = len(active_list)
    if l == 0:
        return 0
    t = 1/l
    index = random.choice(active_list)
    spin = S[tuple(index)]
    neighbor_index = get_neighbor(np.asarray(index),L)
    beforeE = get_energy(spin, index, neighbor_index, S)
    afterE = get_energy(-spin, index, neighbor_index, S)
    deltaE = afterE - beforeE
    if deltaE > 0:
        S[tuple(index)] = -spin
        active_list.remove(tuple(index))
        for nspin in neighbor_index:
            if is_active(nspin,S,L):
                if not (tuple(nspin) in active_list):
                    active_list.append(tuple(nspin))
            else:
                if (tuple(nspin) in active_list):
                    active_list.remove(tuple(nspin))
    return t

In [11]:
def montecarlmethod(S,L,N,t):
    survival1 = 0
    survival2 = 0
    S2 = S.copy()
    S1 = S.copy()
    bonds = dict()
    while True:
        sweep(S1,L,N)
        survival1 = survival1 + 1
        print('S1 Sweep {} done'.format(survival1))
        clear_output(wait=True)
        if survival1 == t:
            break

    S1_active = get_active(S1,L)

    while True:
        k = kineticMonteCarlo(S1,L,S1_active)
        if k == 0:
            break
        survival1 = survival1 + k
    print('S1 done')
    clear_output(wait=True)

    while True:
        sweep(S2,L,N)
        survival2 = survival2 + 1
        print('S2 Sweep {} done'.format(survival2))
        clear_output(wait=True)
        if survival2 == t:
            break

    S2_active = get_active(S2,L)

    while True:
        k = kineticMonteCarlo(S2,L,S2_active)
        if k == 0:
            break
        survival2 = survival2 + k

    return [overlap(S1,S2,N),survival1,survival2] 

In [12]:
bonds = dict()
L = np.array([100, 100])
N = np.prod(L)
model = 'EA'
config = np.random.choice([-1, 1], size=tuple(L))
S2 = config.copy()
S1 = config.copy()

In [13]:
survival1 = 0
survival2 = 0

while len(get_active(S1,L)) != 0:
    sweep(S1,L,N)
    survival1 = survival1 + 1
    if survival1 == 10:
        break

S1_active = get_active(S1,L)

while True:
    k = kineticMonteCarlo(S1,L,S1_active)
    if k == 0:
        break
    survival1 = survival1 + k

while len(get_active(S2,L)) != 0:
    sweep(S2,L,N)
    survival2 = survival2 + 1
    if survival2 == 10:
        break

S2_active = get_active(S2,L)

while True:
    k = kineticMonteCarlo(S2,L,S2_active)
    if k == 0:
        break
    survival2 = survival2 + k

print(overlap(S1,S2,N))
print(survival1)
print(survival2)   

0.4218
10
14.0


In [16]:
L = np.array([100, 35, 35])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N,10))

[0.35622857142857145, 16.999901945753038, 16.74722469238637]


In [17]:
L = np.array([100, 10, 10, 10])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N,10))

[0.3317, 19.901889487517508, 19.112948302026172]


In [18]:
L = np.array([100, 6, 6, 6, 6])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N,10))

[0.3082561728395062, 25.345473053767797, 19.50071685422071]


In [19]:
L = np.array([100, 4, 4, 4, 4, 4])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N,10))

[0.2760546875, 26.82772845118168, 25.217977059365776]


In [20]:
L = np.array([100, 4, 4, 4, 3, 3, 3])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N,10))

[0.27447916666666666, 24.639219521915972, 24.968819439023274]


In [21]:
L = np.array([100, 3, 3, 3, 3, 3, 3, 3])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N,10))

[0.2687334247828075, 32.1945538892262, 28.215191167833108]


In [22]:
L = np.array([100, 3, 3, 3, 3, 3, 3, 2, 2])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N,12))

[0.27997942386831276, 29.6617629189774, 27.14782407125634]


In [23]:
L = np.array([100, 3, 3, 3, 3, 3, 2, 2, 2, 2])
N = np.prod(L)
S = np.random.choice([-1, 1], size=tuple(L))
bonds = dict()
print(montecarlmethod(S,L,N,13))

[0.28444444444444444, 29.010920474282887, 28.42584671490468]
